Login: denis.lusson \
Email: denis.lusson@epita.fr




In [18]:
import numpy as np
import joblib
import os
from sklearn import datasets, linear_model

In [19]:
# Load the diabetes dataset
diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True)

# Split the data into training/testing sets
diabetes_X_train = diabetes_X[:-20]
diabetes_X_test = diabetes_X[-20:]

# Split the targets into training/testing sets
diabetes_y_train = diabetes_y[:-20]
diabetes_y_test = diabetes_y[-20:]

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(diabetes_X_train, diabetes_y_train)

LinearRegression()

In [20]:
joblib.dump(regr, f"regr.joblib");

In [21]:
predict = diabetes_X_test

model = joblib.load(f'regr.joblib')

# Number of thetas
n_thetas = len(model.coef_) + 1

# Get thetas
thetas = f"{model.intercept_}f, "
for coef in model.coef_:
    thetas += str(coef) + "f, "
thetas = thetas[:-2]

# Get Features
features = ""
for row in predict:
  feature = "{"
  for value in row:
    feature += str(value) + ", "
  features += feature[:-2]
  features += "},\n"

n_sample = predict.shape[0]
n_feature = predict.shape[1]

In [29]:
code = f"""\
#include <stdio.h>
float prediction(float *features, float* thetas)
{{
    float res = thetas[0];
    for (int i = 0; i < {len(model.coef_)}; ++i)
        res += features[i] * thetas[i + 1];
    return res;
}}

int main(int argc, char *arvgv[])
{{
    float features[{n_sample}][{n_feature}] = {{{features}}};
    float thetas[{n_thetas}] = {{{thetas}}};

    for (int i = 0; i < {n_sample}; ++i) {{
        printf("%f\\n", prediction(features[i], thetas));
    }}
    
    return 0;
}}
"""

with open("regr.c", "w") as f:
    f.write(code)

print(code)

#include <stdio.h>
float prediction(float *features, float* thetas)
{
    float res = thetas[0];
    for (int i = 0; i < 10; ++i)
        res += features[i] * thetas[i + 1];
    return res;
}

int main(int argc, char *arvgv[])
{
    float features[20][10] = {{-0.0781653239992017, 0.0506801187398187, 0.077863387626902, 0.0528581912385822, 0.0782363059554542, 0.0644472995495832, 0.0265502726256275, -0.00259226199818282, 0.0406722637144977, -0.0093619113301358},
{0.00901559882526763, 0.0506801187398187, -0.0396181284261162, 0.0287580963824284, 0.0383336730676214, 0.0735286049414796, -0.0728539480847234, 0.108111100629544, 0.0155668445407018, -0.0466408735636482},
{0.00175052192322852, 0.0506801187398187, 0.0110390390462862, -0.0194420933298793, -0.0167044412604238, -0.00381906512053488, -0.0470824834561139, 0.0343088588777263, 0.024052583226893, 0.0237749439885419},
{-0.0781653239992017, -0.044641636506989, -0.0406959404999971, -0.081413765817132, -0.100637565610693, -0.112794729823292, 0

In [30]:
!gcc regr.c -O3 -o regr

In [24]:
print('Modèle transpilé :')
!./regr

Modèle transpilé :
197.618469
155.439789
172.886642
111.535378
164.800552
131.069534
259.122375
100.479370
117.060120
124.305038
218.366302
61.198330
132.250443
120.333313
52.544617
194.037964
102.571411
123.566078
211.034622
52.603374


In [31]:
res = regr.predict(diabetes_X_test)
print('Modèle non-transpilé :')
for i in res:
  print(i)

Modèle non-transpilé :
197.61846907503298
155.4397932752124
172.88665147339543
111.53537278539139
164.8005478358925
131.06954874949247
259.1223776097739
100.47935156806848
117.06010519663604
124.30503554810358
218.36632792522653
61.19831283790984
132.25046751101775
120.33329249881926
52.54458691278302
194.03798088336453
102.57139702260844
123.56604986672508
211.034631703066
52.60335674105548
